### ASHRAE - Great Energy Predictor III

In [4]:
import numpy as np
import pandas as pd
import scipy as sp
import copy
import matplotlib as plt
import seaborn as sns
import math as m
import pydot 
import os
import statsmodels.api as sm

### Data reading

In [2]:
datapath= 'C:/Users/vincentlee/Desktop/datascienceprojects/Kaggle/ashrae/'

building_df = pd.read_csv(datapath + 'building_metadata.csv')
weather_train = pd.read_csv(datapath + "weather_train.csv")
weather_test = pd.read_csv(datapath + "weather_test.csv")
train = pd.read_csv(datapath + "train.csv")
test = pd.read_csv(datapath + 'test.csv')



In [3]:
print("train:", train.shape)
print("test:",test.shape)
print("building:",building_df.shape)
print("weathertrain:",weather_train.shape)
print("weathertest:",weather_test.shape)

train: (20216100, 4)
test: (41697600, 4)
building: (1449, 6)
weathertrain: (139773, 9)
weathertest: (277243, 9)


### Creating year,month, weekday and hour columns

for weather_train, weather_test, train and test

#### (i) weather train

In [12]:
from datetime import date
import time

weather_train['timestamp'] = pd.to_datetime(weather_train['timestamp'])
weather_train['year'] = weather_train['timestamp'].map(lambda x: x.strftime('%Y')).astype(int)
weather_train['month'] = weather_train['timestamp'].map(lambda x: x.strftime('%m')).astype(int)
# weekday Monday=0, Sunday=6
weather_train['weekday'] = weather_train['timestamp'].map(lambda x: x.weekday()).astype(int)
weather_train['hour'] = weather_train['timestamp'].map(lambda x: x.hour).astype(int) #24hour

In [18]:
weather_train.head()

,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,year,month,weekday,hour
0,0,2016-01-01 00:00:00,25.0,6.0,20.0,NaN,1019.7,0.0,0.0,2016,1,4,0
1,0,2016-01-01 01:00:00,24.4,NaN,21.1,-1.0,1020.2,70.0,1.5,2016,1,4,1
2,0,2016-01-01 02:00:00,22.8,2.0,21.1,0.0,1020.2,0.0,0.0,2016,1,4,2
3,0,2016-01-01 03:00:00,21.1,2.0,20.6,0.0,1020.1,0.0,0.0,2016,1,4,3
4,0,2016-01-01 04:00:00,20.0,2.0,20.0,-1.0,1020.0,250.0,2.6,2016,1,4,4


#### (ii) weather test

In [15]:
weather_test['timestamp'] = pd.to_datetime(weather_test['timestamp'])
weather_test['year'] = weather_test['timestamp'].map(lambda x: x.strftime('%Y')).astype(int)
weather_test['month'] = weather_test['timestamp'].map(lambda x: x.strftime('%m')).astype(int)
# weekday Monday=0, Sunday=6
weather_test['weekday'] = weather_test['timestamp'].map(lambda x: x.weekday()).astype(int)
weather_test['hour'] = weather_test['timestamp'].map(lambda x: x.hour).astype(int) #24hour

In [19]:
weather_test.head()

,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,year,month,weekday,hour
0,0,2017-01-01 00:00:00,17.8,4.0,11.7,NaN,1021.4,100.0,3.6,2017,1,6,0
1,0,2017-01-01 01:00:00,17.8,2.0,12.8,0.0,1022.0,130.0,3.1,2017,1,6,1
2,0,2017-01-01 02:00:00,16.1,0.0,12.8,0.0,1021.9,140.0,3.1,2017,1,6,2
3,0,2017-01-01 03:00:00,17.2,0.0,13.3,0.0,1022.2,140.0,3.1,2017,1,6,3
4,0,2017-01-01 04:00:00,16.7,2.0,13.3,0.0,1022.3,130.0,2.6,2017,1,6,4


#### train 

In [21]:
# CAUTION: running this code may max out your RAM and may take more than 5 minutes to run this

train['timestamp'] = pd.to_datetime(train['timestamp'])
train['year'] = pd.to_datetime(train['timestamp']).map(lambda x: x.strftime('%Y')).astype(int)
train['month'] = train['timestamp'].map(lambda x: x.strftime('%m')).astype(int)
# weekday Monday=0, Sunday=6
train['weekday'] = train['timestamp'].map(lambda x: x.weekday()).astype(int)
# no hour in train timestamp

In [22]:
train.head()

,building_id,meter,timestamp,meter_reading,year,month,weekday,hour
0,0,0,2016-01-01,0.0,2016,1,4,0
1,1,0,2016-01-01,0.0,2016,1,4,0
2,2,0,2016-01-01,0.0,2016,1,4,0
3,3,0,2016-01-01,0.0,2016,1,4,0
4,4,0,2016-01-01,0.0,2016,1,4,0


#### test

In [23]:
# CAUTION: running this code may max out your RAM and may take more than 5 minutes to run this

test['timestamp'] = pd.to_datetime(test['timestamp'])
test['year'] = pd.to_datetime(test['timestamp']).map(lambda x: x.strftime('%Y')).astype(int)
test['month'] = test['timestamp'].map(lambda x: x.strftime('%m')).astype(int)
test['weekday'] = test['timestamp'].map(lambda x: x.weekday()).astype(int)
# weekday Monday=0, Sunday=6
# no hour in test timestamp

In [26]:
test.head()

,row_id,building_id,meter,timestamp,year,month,weekday,hour
0,0,0,0,2017-01-01,2017,1,6,0
1,1,1,0,2017-01-01,2017,1,6,0
2,2,2,0,2017-01-01,2017,1,6,0
3,3,3,0,2017-01-01,2017,1,6,0
4,4,4,0,2017-01-01,2017,1,6,0


### Reducing memory on dataset

In [24]:
## Reducing memory
## Function to reduce the DF size
def reduce_memory(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
train = reduce_memory(train)
test = reduce_memory(test)
weather_train = reduce_memory(weather_train)
weather_test = reduce_memory(weather_test)
building_df = reduce_memory(building_df)